## Environment Set-up 😀

You have to run it with Nvidia GPU, though...

In [ ]:
# !pip install diffusers==0.14.0 transformers

In [12]:
# !pip install opencv-contrib-python
# !pip install controlnet_aux

In [22]:
# !pip install accelerate

## ControlNet Pipeline 🚀

In [ ]:
import cv2
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import numpy as np
from diffusers.utils import load_image

In [39]:
import cv2
import numpy as np

# Reshape the Image to fit ControlNet model input (512*512)
def re_image(image=None):
    # Check the image
    if image is None:
        print("Error: No image.")
        return

    # Convert the image to a numpy array if it isn't already
    if not isinstance(image, np.ndarray):
        img = np.array(image)
    else:
        img = image

    # Preparing Canvas
    height, width, _ = img.shape
    longer_side = max(height, width)
    square_canvas = 255 * np.ones((longer_side, longer_side, 3), dtype=np.uint8)

    y_offset = (longer_side - height) // 2
    x_offset = (longer_side - width) // 2

    square_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = img

    # Resize the image to 512x512 pixels
    resized_img = cv2.resize(square_canvas, (512, 512), interpolation=cv2.INTER_AREA)

    return resized_img


In [31]:
def Sketch2Paint():

    
    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
    )

    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "Linaqruf/anything-v3.0", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
    )

    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

    # Remove if you do not have xformers installed
    # see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
    # for installation instructions
    # pipe.enable_xformers_memory_efficient_attention()

    pipe.enable_model_cpu_offload()


In [54]:
def generate(img,text,num=20):
    image_re = re_image(image=img)
    image_re0 =Image.fromarray(image_re)

    image = np.array(image_re0)
    low_threshold = 150
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    image = Image.fromarray(image)
    print("starting sketch-to-paint process...")
    image1 = pipe(text, image, num_inference_steps=int(num)).images[0]
    
    return image1

## Use it With GUI! 😺

In [59]:
import ipywidgets as widgets
import PIL.Image
import io

# global variable to store the uploaded image
pub_image = None
pub_num = 20
import ipywidgets as widgets

# create a HTML widget
title = widgets.HTML(
    value="<h2>Sketch-to-Paint Generator By GNK48-Catears 😺</h2>",
)
display(title)


# FileUpload widget
upload = widgets.FileUpload(
    accept='image/*',  # Accept only images
    multiple=False  # Single file
)
display(upload)

# text input field
pubtext = widgets.Text(
    value='best quality, masterpiece, 1girl on the battle field, morden warfare, fallen angels',
    placeholder='Input Your Colorization Text Prompt',
    description='Text Prompt:',
    disabled=False,
    layout=widgets.Layout(width='500px')
)
display(pubtext)

pubnum = widgets.Text(
    value='20',
    placeholder='Input Number of Inference Step (20 is good)',
    description='Step:',
    disabled=False,
    layout=widgets.Layout(width='500px')
)
display(pubnum)

# Generate button
button = widgets.Button(description="Generate")
display(button)

# create an output widget to display the image
output = widgets.Output()
display(output)

# define button click event
def on_button_clicked(b):
    global pub_image 
    global pub_num
   
    output.clear_output()

    text0 = pubtext.value

    # check if an image was uploaded
    if pub_image is None:
        with output:
            print("No image uploaded!")
        return
    
    with output:
        print("Running Sketch-to-Paint Process...")
    # Run ControlNet-Canny
    image1 = generate(img = pub_image, text=text0, num=pub_num)
    output.clear_output()
    
    # display the image
    with output:
        # display(image1)
        print("Image Generated!")
        print(text0)
        display(image1)

# function to be triggered on file upload
def on_file_upload(change):
    global pub_image  # access the global variable
    
    # clear the output widget content
    output.clear_output()

    
    # get the uploaded file
    uploaded_file = upload.value[list(upload.value.keys())[0]]
    image_data = uploaded_file['content']

    # convert to image
    image = PIL.Image.open(io.BytesIO(image_data))
    pub_image = image  # store the image in the global variable
    
    with output:
        display(image)

# add the callback to the 'value' attribute
button.on_click(on_button_clicked)

upload.observe(on_file_upload, 'value')


HTML(value='<h2>Sketch-to-Paint Generator By GNK48-Catears 😺</h2>')

FileUpload(value={}, accept='image/*', description='Upload')

Text(value='best quality, masterpiece, 1girl on the battle field, morden warfare, fallen angels', description=…

Text(value='20', description='Step:', layout=Layout(width='500px'), placeholder='Input Number of Inference Ste…

Button(description='Generate', style=ButtonStyle())

Output()